In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,46.74,76,100,4.74,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,31.66,93,100,29.93,snow
2,2,Port Elizabeth,ZA,-33.9180,25.5701,50.29,81,0,4.61,clear sky
3,3,Chuy,UY,-33.6971,-53.4616,50.83,94,99,4.43,overcast clouds
4,4,Longyearbyen,SJ,78.2186,15.6401,51.64,76,75,1.14,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp =float(input('What is the lowest temperature of a destination you would want to travel to? '))
max_temp =float(input('What is the highest temperature of a destination you would want to travel to? '))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
clean_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
24,24,Kilinochchi,LK,9.3961,80.3982,89.47,58,41,12.21,scattered clouds
37,37,Kirakira,SB,-10.4544,161.9205,83.62,71,66,4.23,broken clouds
42,42,Filingue,NE,14.3521,3.3168,83.05,60,91,6.71,overcast clouds
45,45,Hambantota,LK,6.1241,81.1185,86.34,79,20,13.80,few clouds
49,49,Kidal,ML,18.4411,1.4078,84.18,50,76,9.13,broken clouds
...,...,...,...,...,...,...,...,...,...,...
423,423,Padang,ID,-0.9492,100.3543,84.25,68,93,5.77,overcast clouds
425,425,Sungairaya,ID,0.7000,108.9000,82.09,74,100,5.64,light rain
428,428,Bima,ID,-8.4667,118.7167,88.05,56,97,6.11,overcast clouds
437,437,Dhidhdhoo,MV,6.8833,73.1000,81.52,78,100,18.37,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
for index, row in clean_df.iterrows():
    country_len = len(str(row['Country']))
    
    if country_len == 3:
        clean_df = clean_df.drop(index)
        print(f'Dropped {index}')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
24,Kilinochchi,LK,89.47,scattered clouds,9.3961,80.3982,
37,Kirakira,SB,83.62,broken clouds,-10.4544,161.9205,
42,Filingue,NE,83.05,overcast clouds,14.3521,3.3168,
45,Hambantota,LK,86.34,few clouds,6.1241,81.1185,
49,Kidal,ML,84.18,broken clouds,18.4411,1.4078,
67,Cabo San Lucas,MX,89.62,clear sky,22.8909,-109.9124,
68,Fereydun Kenar,IR,80.58,overcast clouds,36.6864,52.5225,
83,Georgetown,MY,84.13,scattered clouds,5.4112,100.3354,
89,Kuala Terengganu,MY,85.68,broken clouds,5.3302,103.1408,
91,Ishigaki,JP,85.14,clear sky,24.3448,124.1572,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

count = 0

# 6b. Iterate through the hotel DataFrame.

print('Collecting Hotel Information...')

for index,row in clean_df.iterrows():
    
    count += 1
    
    if (count%50 == 0) & (count >=50):
        print('Maximum API request limit,waiting 60 seconds...')
        time.sleep(60)

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except:
        print('Hotel Not Found, skipping...')
        
print('All Done')

Hotel Not Found, skipping...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
All Done


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
for index, row in hotel_df.iterrows():
    hotel_name = len(row['Hotel Name'])
    
    if hotel_name == 0:
        hotel_df = hotel_df.drop(index)

In [11]:
clean_hotel_df = hotel_df.copy()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>and<dd>{Max Temp} °F</dd> 
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
hotel_marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(hotel_marker_layer)
# 11b. Display the figure
fig



AttributeError: module 'collections' has no attribute 'Iterable'